In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight : bold;}
div.input {font-family:Consolas; font-size:12pt;}
</style>
"""))

In [2]:
class User:
    def __init__(self, name, tel, age, grade, mail, etc):
        self.name = name
        self.tel = tel
        self.age = age
        self.grade = grade
        self.mail = mail
        self.etc = etc
    
    def as_dict(self):
        return {'name':self.name, 'tel':self.tel, 'age':self.age,
                'grade':self.grade, 'mail':self.mail, 'etc':self.etc}
    
    def __str__(self):
        return "{:3}\t{:13}\t{:3}\t{:>5}\t{:15}\t{}".format(self.name, self.tel, self.age,
                                                            self.grade, self.mail, self.etc)

In [4]:
# 1. 입력
# 사용자로부터 '이름,전화번호,나이,등급,메일,기타' 를 입력 받아 db에 추가
def insert_member():
    global conn
    cursor = conn.cursor()
    mname = input("이름 입력 :")
    tel = input("전화번호 입력 :")
    try:
        age = int(input("나이 입력 :"))
        if age > 150:
            print('입력을 기준을 초과하여 100살로 임의 설정')
            age = 100
        elif age <= 0:
            print('0이하로 입력하여 1살로 임의 설정')
            age = 1
    except ValueError as v:
        print('잘못된 입력이므로 30살로 임의 설정합니다.')
        age = 30
    try:
        grade = int(input("등급(1~5) 입력 :"))
        if grade > 5:
            grade = 5
            print('5이상을 입력하여 5등급으로 임의 설정.')
        elif grade < 1:
            grade = 1
            print('1미만을 입력하여 1등급으로 임의 설정.')
    except ValueError:
        grade = int(input("등급(1~5) 입력 :"))
        print('잘못된 입력이므로 3등급으로 설정합니다.')
        grade = 3
    mail = input("메일 입력 :")
    etc = input('특징 입력 :')
    
    data = {'mname':mname, 'tel':tel, 'age':age, 'grade':grade, 'mail':mail, 'etc':etc}
    cursor.execute("INSERT INTO MEMBER VALUES(:mname, :tel, :age, :grade, :mail, :etc)", data)

In [29]:
# 2 전체출력
# select * from member 의 결과를 출력, 데이터프레임 혹은 리스트 형태로 출력해도 됨
def select_all():
    global conn
    cursor = conn.cursor()
    
    sql = "SELECT * FROM MEMBER"
    cursor.execute(sql)
    
    data = cursor.fetchall()
    data_df = pd.DataFrame(data) # 데이터프레임
    data_df.columns = [row[0] for row in cursor.description]
    
    if len(data_df) == 0:
        print('출력할 데이터가 없습니다.')
    else:
        print(data_df)

In [19]:
# 3 이름 검색
# 사용자로부터 검색하고자하는 이름을 받아 결과 출력
def select_name():
    global conn
    cursor = conn.cursor()
    select_user = input('검색 할 이름 입력 :')
    sql = "SELECT * FROM MEMBER WHERE MNAME = :mname"
    
    cursor.execute(sql, {'mname':select_user})
    member = cursor.fetchall()
                   
    if len(member) == 0:
        print('입력하신 이름의 데이터는 없습니다')
    else:
        for e in member:
            print(e)

In [36]:
# 4 메일 삭제
# 사용자로부터 삭제할 이메일을 입력받아 해당 이메일이 있는지 확인, 있으면 삭제 및 메세지 출력
# 없으면 없다는 메세지 출력
def delete_mail():
    global conn
    cursor = conn.cursor()
    
    del_mail = input('삭제 할 메일 입력 :')

    cursor.execute("SELECT * FROM MEMBER")
    members = cursor.fetchall()

    if len(members) == 0:
        print('해당 메일이 없습니다.')
    else:
        print('해당 메일이 존재합니다. 삭제를 진행합니다')
        cursor.execute("DELETE FROM MEMBER WHERE MAIL = :mail", {'mail':del_mail})

In [32]:
# 5 csv 내보내기
# select * from member의 결과를 csv로 내보내기 (헤더 포함)
def save_csv():
    import csv
    global conn
    cursor = conn.cursor()
    customer_dict_list = []
    result = cursor.execute("SELECT * FROM MEMBER")
    
    fieldnames = ["mname","tel","age","grade","mail","etc"] # csv의 header
    filename = input('저장할 파일명 입력 :')
    
    try:
        with open('Data/'+filename+'.csv', 'w', encoding = 'CP949', newline = '') as f:
            writer = csv.writer(f)
            writer.writerow([row[0] for row in cursor.description]) # write headers
            writer.writerows(cursor)
    except FileNotFoundError as e:
        print(e)

In [9]:
# 0 종료
def close_sql():
    global conn
    print('종료합니다.')
    conn.commit()
    conn.close()

In [10]:
# MAIN 함수
def main():
    while True:
        print('1 : 입력 | 2 : 전체출력 | 3 : 이름 검색\n4 : 메일 삭제 | 5 : 내보내기(csv) | 0 : 종료')
        menu = int(input('메뉴 선택 :'))
        # 입력받은 번호로 customer 객체를 반환
        if menu == 1:
            insert_member()
        elif menu == 2:
            select_all()
        elif menu == 3:
            select_name()
        elif menu == 4:
            delete_mail()
        elif menu == 5:
            save_csv()
        elif menu == 0:
            close_sql()
            break

In [38]:
if __name__ == '__main__':
    import cx_Oracle
    import pandas as pd
    oracle_dsn = cx_Oracle.makedsn(host = 'localhost', port = 1521, sid = 'xe')    
    
    global conn
    conn = cx_Oracle.connect("scott", "tiger", dsn = oracle_dsn)
    
    main()

1 : 입력 | 2 : 전체출력 | 3 : 이름 검색
4 : 메일 삭제 | 5 : 내보내기(csv) | 0 : 종료
메뉴 선택 :2
  MNAME            TEL  AGE  GRADE       MAIL   ETC
0   신길동  010-3333-3333   28      5  ooo@o.com    착함
1    장비  010-2222-2222   29      5  jjj@j.com    멍청
2    손책  010-3333-3333   26      4  qqq@q.com  카리스마
1 : 입력 | 2 : 전체출력 | 3 : 이름 검색
4 : 메일 삭제 | 5 : 내보내기(csv) | 0 : 종료
메뉴 선택 :3
검색 할 이름 입력 :장비
('장비', '010-2222-2222', 29, 5, 'jjj@j.com', '멍청')
1 : 입력 | 2 : 전체출력 | 3 : 이름 검색
4 : 메일 삭제 | 5 : 내보내기(csv) | 0 : 종료
메뉴 선택 :4
삭제 할 메일 입력 :ooo@o.com
해당 메일이 존재합니다. 삭제를 진행합니다
1 : 입력 | 2 : 전체출력 | 3 : 이름 검색
4 : 메일 삭제 | 5 : 내보내기(csv) | 0 : 종료
메뉴 선택 :2
  MNAME            TEL  AGE  GRADE       MAIL   ETC
0    장비  010-2222-2222   29      5  jjj@j.com    멍청
1    손책  010-3333-3333   26      4  qqq@q.com  카리스마
1 : 입력 | 2 : 전체출력 | 3 : 이름 검색
4 : 메일 삭제 | 5 : 내보내기(csv) | 0 : 종료
메뉴 선택 :5
저장할 파일명 입력 :ksw_oracle
1 : 입력 | 2 : 전체출력 | 3 : 이름 검색
4 : 메일 삭제 | 5 : 내보내기(csv) | 0 : 종료
메뉴 선택 :0
종료합니다.
